In [1]:
import numpy as np
import pandas as pd

In [2]:
%%time
train_df = pd.read_csv("../../01_input/train.csv")
test_df = pd.read_csv("../../01_input/test.csv")

CPU times: user 12.2 s, sys: 460 ms, total: 12.7 s
Wall time: 12.7 s


In [3]:
submit_flg=1 #全件処理かどうかをコントロール
grid_flg=1 #GridSearchをするかコントロール
SEED=12345
sample_num=10000
fold_num=5

#train関連
train_drop_col=['ID_code','target']
train_label='target'

#test関連
test_drop_col=['ID_code']

In [4]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
x_test=test_df.drop(test_drop_col,axis=1)

特徴量作成コードを記載

In [14]:
pd.set_option('display.max_columns', 500)
x_train.describe()


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.0000

In [15]:
new_feature_train=pd.DataFrame()
new_feature_test=pd.DataFrame()
for i in range(-30,30):  
    train_negcount = (x_train < i)
    test_negcount = (x_test < i)
    train_poscount = (x_train > i)
    test_poscount = (x_test > i)
    new_feature_train['NegCount_'+str(i)]=train_negcount.sum(axis=1)
    new_feature_test['NegCount_'+str(i)]=test_negcount.sum(axis=1)
    new_feature_train['PosCount_'+str(i)]=train_poscount.sum(axis=1)
    new_feature_test['PosCount_'+str(i)]=test_poscount.sum(axis=1)

In [16]:
new_feature_train.head()

,NegCount_-30,PosCount_-30,NegCount_-29,PosCount_-29,NegCount_-28,PosCount_-28,NegCount_-27,PosCount_-27,NegCount_-26,PosCount_-26,NegCount_-25,PosCount_-25,NegCount_-24,PosCount_-24,NegCount_-23,PosCount_-23,NegCount_-22,PosCount_-22,NegCount_-21,PosCount_-21,NegCount_-20,PosCount_-20,NegCount_-19,PosCount_-19,NegCount_-18,PosCount_-18,NegCount_-17,PosCount_-17,NegCount_-16,PosCount_-16,NegCount_-15,PosCount_-15,NegCount_-14,PosCount_-14,NegCount_-13,PosCount_-13,NegCount_-12,PosCount_-12,NegCount_-11,PosCount_-11,NegCount_-10,PosCount_-10,NegCount_-9,PosCount_-9,NegCount_-8,PosCount_-8,NegCount_-7,PosCount_-7,NegCount_-6,PosCount_-6,NegCount_-5,PosCount_-5,NegCount_-4,PosCount_-4,NegCount_-3,PosCount_-3,NegCount_-2,PosCount_-2,NegCount_-1,PosCount_-1,NegCount_0,PosCount_0,NegCount_1,PosCount_1,NegCount_2,PosCount_2,NegCount_3,PosCount_3,NegCount_4,PosCount_4,NegCount_5,PosCount_5,NegCount_6,PosCount_6,NegCount_7,PosCount_7,NegCount_8,PosCount_8,NegCount_9,PosCount_9,NegCount_10,PosCount_10,NegCount_11,PosCount_11,NegCount_12,PosCount_12,NegCount_13,PosCount_13,NegCount_14,PosCount_14,NegCount_15,PosCount_15,NegCount_16,PosCount_16,NegCount_17,PosCount_17,NegCount_18,PosCount_18,NegCount_19,PosCount_19,NegCount_20,PosCount_20,NegCount_21,PosCount_21,NegCount_22,PosCount_22,NegCount_23,PosCount_23,NegCount_24,PosCount_24,NegCount_25,PosCount_25,NegCount_26,PosCount_26,NegCount_27,PosCount_27,NegCount_28,PosCount_28,NegCount_29,PosCount_29
0,0,200,0,200,0,200,0,200,0,200,0,200,0,200,0,200,0,200,1,199,1,199,2,198,2,198,2,198,3,197,3,197,4,196,6,194,6,194,6,194,7,193,8,192,10,190,12,188,13,187,16,184,20,180,23,177,25,175,28,172,36,164,44,156,50,150,62,138,71,129,78,122,95,105,101,99,107,93,116,84,126,74,131,69,142,58,151,49,157,43,165,35,169,31,177,23,181,19,186,14,187,13,188,12,190,10,191,9,192,8,193,7,194,6,195,5,195,5,195,5
1,1,199,1,199,1,199,1,199,1,199,1,199,1,199,1,199,1,199,1,199,1,199,1,199,1,199,1,199,1,199,3,197,4,196,6,194,6,194,9,191,9,191,11,189,13,187,16,184,16,184,19,181,24,176,24,176,25,175,28,172,33,167,43,157,53,147,61,139,73,127,75,125,90,110,95,105,112,88,127,73,134,66,140,60,149,51,155,45,165,35,168,32,172,28,177,23,177,23,181,19,183,17,185,15,185,15,188,12,189,11,190,10,191,9,192,8,192,8,193,7
2,0,200,0,200,0,200,0,200,0,200,0,200,0,200,0,200,1,199,1,199,1,199,1,199,1,199,2,198,2,198,2,198,2,198,4,196,4,196,7,193,8,192,10,190,10,190,13,187,15,185,21,179,24,176,25,175,31,169,36,164,42,158,50,150,58,142,67,133,78,122,88,112,101,99,110,90,121,79,129,71,136,64,143,57,150,50,159,41,162,38,174,26,178,22,178,22,181,19,182,18,187,13,192,8,194,6,194,6,196,4,197,3,198,2,198,2,198,2,199,1
3,1,199,1,199,1,199,1,199,1,199,2,198,3,197,3,197,3,197,3,197,3,197,4,196,4,196,4,196,4,196,5,195,5,195,5,195,6,194,7,193,9,191,9,191,11,189,13,187,14,186,17,183,18,182,21,179,28,172,35,165,39,161,46,154,58,142,69,131,77,123,86,114,96,104,102,98,113,87,124,76,129,71,138,62,148,52,159,41,164,36,172,28,178,22,178,22,183,17,188,12,189,11,190,10,190,10,191,9,194,6,196,4,196,4,196,4,197,3,197,3
4,2,198,2,198,2,198,3,197,3,197,3,197,3,197,3,197,4,196,4,196,4,196,4,196,4,196,4,196,4,196,4,196,4,196,4,196,7,193,7,193,7,193,9,191,11,189,13,187,15,185,16,184,19,181,25,175,26,174,32,168,39,161,45,155,56,144,63,137,70,130,74,126,90,110,101,99,114,86,119,81,130,70,136,64,143,57,151,49,161,39,170,30,171,29,175,25,178,22,181,19,187,13,189,11,191,9,191,9,192,8,192,8,194,6,194,6,195,5,196,4


In [17]:
new_feature_train.to_csv('../../02_feature/119_train_NegPosCount.csv',index=False,header=True)
new_feature_test.to_csv('../../02_feature/119_test_NegPosCount.csv',index=False,header=True)